In [1]:
import os

In [28]:
folder = 'questions'
files = [file for file in os.listdir(folder) if file.endswith('.txt')]

print(len(files))

7846


In [29]:
questions = []

for file in files:
    with open(os.path.join(folder, file)) as f:
        questions.append(f.read())

In [30]:
with open('raw_questions.txt', 'w') as f:
    f.write('\n'.join(questions))

In [31]:
import markovify

In [32]:
mc = markovify.Text('\n'.join(questions), state_size = 2)

In [34]:
for _ in range(50): print(mc.make_sentence(), '\n')

как на нее управу Как выйти из состава российской империи? 

Почему женщины любят ездить на ней по городу? 

Девушки, какими качествами должен обладать мужчина, чтобы вы приняли чувства и ответили на них? 

None 

Проверти знаки препинания и если да то где? 

Как вернуть молчела если он незнает русского только может он спрасить сколько тебе лет и чего им здесь надо, чего хотят? 

Я не могу раскрутить. 

None 

скачал программу adobe reader 7 с родного официального сайта поддержки будет ли встреча весною с той, что мне нравятся мне отказывают ? Выерите в русалок?И знаете ли что скоро сигареты подорожают? 

за 3 дня 10кг похудеть можно ли найти любовь как понять себя? 

Как похудеть за короткое время достичь оргазма? 

Я не хвастаюсь но с приличной долей юмора! 

что означает имя Резеда Где найти хорошего фотографа? 

Скажите пожалуйста все названия всех аниме, находящихся в интро Может ли кто-нибудь посодействовать в размещении песни в ротацию на радио? 

Найти книги Энн Райс >Лэшер >Та